In [1]:
import cv2
import numpy as np
import os,glob
import torch


from learn import CustomResNet,SimpleCNN,SimpleCNN_1

In [2]:
def apply_hsv_threshold(image):
    low_h, high_h = 90, 100  # Example range for yellow hue
    low_s, high_s = 140, 255 # Example range for saturation
    low_v, high_v = 0, 255 # Example range for value
    

    # Convert the image from RGB to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Define the lower and upper bounds of the HSV threshold
    lower_bound = np.array([low_h, low_s, low_v])
    upper_bound = np.array([high_h, high_s, high_v])
    
    # Create a mask where pixels within the threshold are white, and others are black
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Create an all black image
    black_image = np.zeros_like(image)
    
    # Copy the pixels from the original image where the mask is white
    result_image = np.where(mask[:, :, None] == 255, image, black_image)
    
    return result_image

In [3]:
input_directory = "dataset/collect"
output_directory = "dataset/label/"

entries = os.listdir(input_directory)

# Filter to include only .jpg files
img_files = [file for file in entries if file.endswith('.jpg')]

In [4]:
img_files

['sample0.jpg', 'sample3.jpg', 'sample2.jpg']

In [ ]:
# デバイスの設定（CUDAが利用可能な場合はGPUを使用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weight = 'weight/heavy/model_best.pth'
# モデルのインスタンスを作成
model = SimpleCNN()
# モデルの重みをロードし、推論モードに設定
model.load_state_dict(torch.load(weight, map_location=device))
model.to(device)
model.eval()

In [5]:
input_directory = 'dataset/collect/'
output_path = 'dataset/label/'

for img in img_files:
    img_path = input_directory + img
    image = cv2.imread(img_path)

    mask_image = apply_hsv_threshold(image)

    #モデルにmask=imageを入力し、出力から座標、半径を得る
    #ファイルの名前を、filename = '_x_y_r_uuid.jpg'として保存
    
    